In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

df = pd.read_csv("../project_3_final_table.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'caseid_new', 'children_in_hh', 'age_difference',
       'race_gap', 'religious_gap', 'edu_gap', 'met_online', 'met_at_work',
       'met_at_school', 'met_at_church', 'met_travel', 'met_social',
       'met_party', 'met_f_and_f', 'met_as_neighbors', 'met_public_space',
       'met_offline_dating', 'met_other', 'how_met_count', 'relationship_len',
       'partner_deceased', 'married_w1_married', 'married_w1_not married',
       'age_gap_bin_0 to 3', 'age_gap_bin_11 to 20', 'age_gap_bin_4 to 5',
       'age_gap_bin_6 to 10', 'age_gap_bin_>=20',
       'same_sex_couple_different sex couple',
       'same_sex_couple_same-sex couple', 'edu_gap_bin_0 to 2',
       'edu_gap_bin_3 to 4', 'edu_gap_bin_5 to 6', 'edu_gap_bin_>6',
       'parental_approval_approve', 'parental_approval_dont_approve',
       'relationship_quality_w1_excellent', 'relationship_quality_w1_fair',
       'relationship_quality_w1_good', 'relationship_quality_w1_poor',
       'relationship_quality_w1_v

In [4]:
# Reverse get_dummies
rev_how_met = df[['caseid_new','met_online', 'met_at_work', 'met_at_school',
       'met_at_church', 'met_travel', 'met_social', 'met_party', 'met_f_and_f',
       'met_as_neighbors', 'met_public_space', 'met_offline_dating',
       'met_other']]

rev_how_met.set_index('caseid_new', inplace=True)

rev_how_met = rev_how_met[rev_how_met==1].stack().reset_index().drop(0,1)

rev_how_met = rev_how_met.rename(columns={"level_1" : "how_met"})

rev_how_met["how_met"] = rev_how_met["how_met"].map({'met_online':0, 'met_at_work' : 1, 'met_at_school' :2,
       'met_at_church' :3 , 'met_travel':4, 'met_social':5, 'met_party':6, 'met_f_and_f':7,
       'met_as_neighbors':8, 'met_public_space': 9, 'met_offline_dating':10,
       'met_other':11})

df = df.merge(rev_how_met, on="caseid_new")

df

,Unnamed: 0,caseid_new,children_in_hh,age_difference,race_gap,religious_gap,edu_gap,met_online,met_at_work,met_at_school,...,region_west,partner_political,resondent_political,income_gap_raw,same_political,income_gap,respondent_earn_more,relationship_quality_w1,relationship_len_cat,how_met
0,0,22526,0,4.0,1,1,2.0,1,0,0,...,0,democrat,democrat,partner earned more,1,1.0,0.0,good,second_quartile,0
1,0,22526,0,4.0,1,1,2.0,1,0,0,...,0,democrat,democrat,partner earned more,1,1.0,0.0,good,second_quartile,9
2,1,23286,0,2.0,0,1,1.0,0,0,1,...,1,democrat,democrat,i earned more,1,1.0,1.0,good,second_quartile,2
3,1,23286,0,2.0,0,1,1.0,0,0,1,...,1,democrat,democrat,i earned more,1,1.0,1.0,good,second_quartile,7
4,1,23286,0,2.0,0,1,1.0,0,0,1,...,1,democrat,democrat,i earned more,1,1.0,1.0,good,second_quartile,8
5,2,26315,0,9.0,0,1,2.0,1,0,0,...,0,democrat,democrat,i earned more,1,1.0,1.0,good,bottom_quartile,0
6,2,26315,0,9.0,0,1,2.0,1,0,0,...,0,democrat,democrat,i earned more,1,1.0,1.0,good,bottom_quartile,7
7,3,28536,0,2.0,0,1,1.0,0,0,0,...,0,democrat,democrat,i earned more,1,1.0,1.0,good,second_quartile,9
8,4,29584,0,7.0,0,0,0.0,0,1,0,...,0,democrat,democrat,partner earned more,1,1.0,0.0,good,top_quartile,1
9,5,31456,0,0.0,0,1,3.0,1,0,0,...,0,independent,democrat,partner earned more,0,1.0,0.0,excellent,bottom_quartile,0


In [5]:
df['family_size'].fillna(round(df['family_size'].mean()), inplace=True)
df['relationship_len'].fillna(round(df['relationship_len'].mean()), inplace=True)
df['age_when_met'].fillna(round(df['age_when_met'].mean()), inplace=True)
df['region'].fillna(df['region'].mode(), inplace=True)
df['metro'].fillna(df['metro'].mode(), inplace=True)
df['same_political'].fillna(df['same_political'].mode(), inplace=True)
df['income_gap'].fillna(round(df['income_gap'].mean()), inplace=True)
df['partner_deceased'].fillna(df['partner_deceased'].mode(), inplace=True)
df['relationship_quality_w1'].fillna(df['relationship_quality_w1'].mode()[0], inplace=True)
df['age_difference'].fillna(round(df['age_difference'].mean()), inplace=True)
df['edu_gap'].fillna(round(df['edu_gap'].mean()), inplace=True)
df['respondent_earn_more'].fillna(round(df['respondent_earn_more'].mean()), inplace=True)


df.relationship_quality_w1_numerical = df.relationship_quality_w1.map({'very poor':0, 'poor':1, 'fair':2, 'good':3, 'excellent':4})
# df['relationship_quality_w1_numerical'].fillna(round(df['relationship_quality_w1_numerical'].mean()), inplace=True)

C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app


### Predicted Length of Relationship

In [6]:
selected_features = df.drop(["Unnamed: 0", "caseid_new", "relationship_len_cat", "relationship_len", "partner_political", "resondent_political", "income_gap_raw", "relationship_quality_w1_excellent", "relationship_quality_w1_good", "relationship_quality_w1_poor", 
        "relationship_quality_w1_fair", "relationship_quality_w1_very poor", "respondent_earn_more", "partner_deceased", "how_met_count", "married_w1_not married", "same_sex_couple_same-sex couple", "parental_approval_dont_approve", 'region_midwest', 'region_northeast', 'region_south', 'region_west', 'edu_gap_bin_0 to 2', 
       'edu_gap_bin_3 to 4', 'edu_gap_bin_5 to 6', 'edu_gap_bin_>6','age_gap_bin_0 to 3', 'age_gap_bin_11 to 20', 'age_gap_bin_4 to 5',
       'age_gap_bin_6 to 10', 'age_gap_bin_>=20', "relationship_quality_w1", 'met_online', 'met_at_work', 'met_at_school',
       'met_at_church', 'met_travel', 'met_social', 'met_party', 'met_f_and_f',
       'met_as_neighbors', 'met_public_space', 'met_offline_dating',
       'met_other'], axis=1)
y1 = df["relationship_len_cat"]
# y = df["relationship_quality_w1"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, y1, random_state=0)

In [7]:
selected_features.columns

Index(['children_in_hh', 'age_difference', 'race_gap', 'religious_gap',
       'edu_gap', 'married_w1_married', 'same_sex_couple_different sex couple',
       'parental_approval_approve', 'age_when_met', 'family_size', 'region',
       'metro', 'same_political', 'income_gap', 'how_met'],
      dtype='object')

In [8]:
# Creating a random forest classifier
# model1 = RandomForestClassifier(n_jobs=2, max_depth=20, random_state=0, min_samples_leaf=25, oob_score=True)
model1 = RandomForestClassifier(n_jobs=2, random_state=0)

# Training classifier
model1.fit(X_train, y_train)


C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [9]:
# Finding important features

num_important = (model1.feature_importances_).astype(str)
features = X_test.columns

def myfunc(a, b):
    return [round(float(b)*100, 2)] + [a]

def to_percentage(score):
    return f'{round(score * 100, 2)}%'

important_features_map = map(myfunc, features, num_important)
important_features = dict(important_features_map)
 
sorted_important_features = sorted(important_features.items(), reverse=True)

print(f"Training Data Score: {to_percentage(model1.score(X_train, y_train))}")
print(f"Testing Data Score: {to_percentage(model1.score(X_test, y_test))}")
print("-----------------------------------------------------")
print(f"Most important features:\n{sorted_important_features[:5]}")

Training Data Score: 99.4%
Testing Data Score: 76.84%
-----------------------------------------------------
Most important features:
[(17.89, 'age_when_met'), (11.95, 'age_difference'), (10.32, 'family_size'), (10.18, 'married_w1_married'), (8.67, 'edu_gap')]


In [10]:
preds = model1.predict(X_test)
pd.crosstab(y_test, preds, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,bottom_quartile,second_quartile,third_quartile,top_quartile
Actual Result,,,,
bottom_quartile,287,26,9,7
second_quartile,39,239,42,5
third_quartile,35,48,216,24
top_quartile,17,34,22,280


## Predict Relationship Quality

In [11]:
y2 = df["relationship_quality_w1"]


X_train, X_test, y_train, y_test = train_test_split(selected_features, y2, random_state=0)

In [12]:
# Creating a random forest classifier
model2 = RandomForestClassifier(n_jobs=2, random_state=0)
# Training classifier
model2.fit(X_train, y_train)
# Finding important features

num_important = (model2.feature_importances_).astype(str)
features = X_test.columns

def myfunc(a, b):
    return [round(float(b)*100, 2)] + [a]

def to_percentage(score):
    return f'{round(score * 100, 2)}%'

important_features_map = map(myfunc, features, num_important)
important_features = dict(important_features_map)
 
sorted_important_features = sorted(important_features.items(), reverse=True)

print(f"Training Data Score: {to_percentage(model2.score(X_train, y_train))}")
print(f"Testing Data Score: {to_percentage(model2.score(X_test, y_test))}")
print("-----------------------------------------------------")
print(f"Most important features:\n{sorted_important_features[:5]}")

C:\Users\megam\Anaconda3\envs\PythonData\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training Data Score: 98.75%
Testing Data Score: 74.44%
-----------------------------------------------------
Most important features:
[(18.64, 'age_when_met'), (14.04, 'age_difference'), (13.53, 'family_size'), (10.45, 'edu_gap'), (9.01, 'how_met')]


In [13]:
preds2 = model2.predict(X_train)
y_test_predicted_full_trees = model2.predict(X_test)

pd.crosstab(y_test, y_test_predicted_full_trees, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,excellent,fair,good,poor,very poor
Actual Result,,,,,
excellent,732,4,64,0,0
fair,57,40,13,0,0
good,177,3,205,0,1
poor,11,0,5,9,0
very poor,3,0,2,0,4


In [14]:
import joblib
length_model = 'rel_length_RF.sav'
quality_model = 'rel_quality_model_RF.sav'

joblib.dump(model1, length_model)
joblib.dump(model2, quality_model)

['rel_quality_model_RF.sav']